In [7]:
def sequence_extractor(Species, Scaff, reverse_c, start, end, frame = 1, trans = 1 ):
    from Bio import SeqIO
    from Bio.Seq import Seq
    import os
    
    out_Seq = ''
    
    
    #Genome folder
    entries = os.listdir("/mnt/f/Genomes_2023-12-26/"+Species)
    
    #Get genome file from Genome folder
    for file_names  in entries:
        if ".nhr" in file_names:
            Genome_name = file_names[:-4]
            break
    
    #Read the genome file
    fasta_file = open(("/mnt/f/Genomes_2023-12-26/"+Species+"/"+Genome_name),'r')
    
    #Extract the sequence 
    for record in SeqIO.parse(fasta_file,"fasta"):
        if record.id == Scaff:
            sequence = str(record.seq)
            out_Seq = Seq(sequence[start-1:end])
            if reverse_c == 1:
                out_Seq = out_Seq.reverse_complement()
                
    if len(out_Seq) < 10000: #fixing error due to mistake in typing the coordinate, change this for longer sequence
#        print (out_Seq)
        if frame == 1 :
            out_trans = out_Seq[0:]
        if frame == 2 :
            out_trans = out_Seq[1:]
        if frame == 3 :
            out_trans = out_Seq[2:]
    else:
        print ("too long")
        assert(False)
        
        
    #    break
    fasta_file.close()
    if trans == 1:
        return (out_trans.translate())
    else:
        return(out_Seq)
#

In [8]:
import os
from Bio import SeqIO

In [11]:
#%reset -f
blast_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/3.Satyrine"
query_species = "5.Bicyclus_anynana"
query_transcript = "XM_024088150.2"
#Get Species list, the list may be a list of files or a single species
query_species = "5.Bicyclus_anynana"
# phylo_species = open("C:/Users/sauba/Desktop/Work_Stuff/MRJP/0.for_automation_temp_files/phylo_species.txt",'r')
# species_list = phylo_species.readlines()
# species_list = ["10.Plutella_xylostella","11.Bombyx_mori","12.Manduca_sexta","13.Danaus_plexippus","14.Zerene_cesonia","15.Hyposmocoma_kahamanoa","16.Papilio_xuthus","17.Papilio_polytes","18.Bombyx_mandarina","19.Trichoplusia_ni","2.Galleria_mellonella","20.Vanessa_tameamea","21.Amyelois_transitella","22.Pieris_brassicae","23.Vanessa_atalanta","24.Spodoptera_frugiperda","25.Pieris_napi","26.Vanessa_cardui","27.Maniola_hyperantus","28.Maniola_jurtina","29.Nymphalis_io","3.Spodoptera_litura","30.Leptidea_sinapis","31.Leguminivora_glycinivorella","32.Colias_croceus","33.Pararge_aegeria","36.Melitaea_cinxia","37.Aricia_agestis","38.Papilio_machaon","39.Leptidea_sinapis","4.Helicoverpa_armigera","40.Colias_croceus","41.Manduca_sexta","42.Spodoptera_frugiperda","43.Ostrinia_furnacalis","5.Bicyclus_anynana","6.Pectinophora_gossypiella","7.Helicoverpa_zea","8.Plodia_interpunctella","9.Pieris_rapae"]
species_list = os.listdir(f"{blast_location}/1.Blast_result")
# species_list.remove(query_species)
# species_list = ["5.Bicyclus_anynana"]
# species_list = ["29.Nymphalis_io","3.Spodoptera_litura","30.Leptidea_sinapis","31.Leguminivora_glycinivorella","32.Colias_croceus","33.Pararge_aegeria","36.Melitaea_cinxia","37.Aricia_agestis","38.Papilio_machaon","39.Leptidea_sinapis","4.Helicoverpa_armigera","40.Colias_croceus","41.Manduca_sexta","42.Spodoptera_frugiperda","43.Ostrinia_furnacalis","5.Bicyclus_anynana","6.Pectinophora_gossypiella","7.Helicoverpa_zea","8.Plodia_interpunctella","9.Pieris_rapae"]
# species_list = ["36.Melitaea_cinxia","37.Aricia_agestis","38.Papilio_machaon","39.Leptidea_sinapis","4.Helicoverpa_armigera","40.Colias_croceus","41.Manduca_sexta","42.Spodoptera_frugiperda","43.Ostrinia_furnacalis","5.Bicyclus_anynana","6.Pectinophora_gossypiella","7.Helicoverpa_zea","8.Plodia_interpunctella","9.Pieris_rapae"]
#print(species_list)
# species_list = ["3.Spodoptera_litura"]
#get list of query names

query_file = SeqIO.parse(f"{blast_location}/1.Query/{query_species}/{query_transcript}/query_protein.fa", 'fasta')
query_name_list = []

for records in query_file:
    query_name_list.append(records.id)



for species in species_list:
    # print(species_name)
    # species = species_name.split(".")[1]
    header = "Species," + "Scaffold," + "Start," + "Stop," + "Complement," + "Error," + "Gene,"+ "Query_start," + "Query_stop,"+ "Query_Length\n" 
    Output_Sequence = header
    scaff = "Intial_value"
    scaff_old = "Intial_value"
    old_end = 0
    
#Run for each query in the query list
    for i in range(len(query_name_list)):
        query_name = query_name_list[i]
        Length_switch = "0"
        with open(f"{blast_location}/1.Blast_result/{species}/{species}_blast_out.txt",'r') as tblast_out:
            lines_in_file = tblast_out.readlines()
        #print(lines_in_file)

        result_section_switch = 0
        start_coor_switch = 0
        query_start_coor_switch = 0
        stop_coor_switch = 0
        error = "N"
        break_switch = 0

        start = 0
        stop = 0
        start_coor = 0
        stop_coor = 0
        query_length = 0

        for lines in lines_in_file:

#             print(lines)
            if query_name in lines:
            #Initialize that results can now be checked
                result_section_switch = 1
                query_species_split = lines.split(" ")[1].split("_")
                query_species = str(query_species_split[0].split(".")[1]+"_"+query_species_split[1].rstrip())
                print(query_species)
            if result_section_switch == 1 and "Lambda" in lines:
            #This block indicates end of the results block in blast output
                result_section_switch == 0
                
                break

            if result_section_switch == 1:
            #While checking the result
                if "Length=" in lines and Length_switch == "0":
                #Get query length from the blast output
                    
                    query_length = int(lines.split("=")[1].rstrip())
                    
                    Length_switch = 1 #Indicated length has been acquired
                    
                if ("Score" in lines or ">" in lines) and (start_coor_switch == 1):
    #                print (lines)
                    break
        
                if ">" in lines:
                #Start of the first result
                    scaff = lines.split(" ")[0][1:] #Scaffold from the result
                    if scaff_old != "Intial_value" and scaff_old != scaff:
                        error = "Y"
                    scaff_old = scaff
                    
                if "Query" in lines and "=" not in lines:
                #Read the query line in output
                    if query_start_coor_switch == 0:
#                        print(lines)
                        query_start_coor = int(lines.split(" ")[2])
                        query_start_coor_switch = 1
                        #Query start coordinate fixed
            
                    query_stop_coor =int(lines.split(" ")[-1][:-1])
                    #Keep getting query stop coordinates for multiline result
    #                print (stop_coor)
                    
                if "Sbjct" in lines:
                #Read the blast target line
                    if start_coor_switch == 0:
                        start_coor = int(lines.split(" ")[2])
                        start_coor_switch = 1
                    stop_coor =int(lines.split(" ")[-1][:-1])
                    #Keep getting target stop coordinates for multiline result
                
                

        
        if break_switch == 1:
            break

        if start_coor < stop_coor:
            complement = "0" #Forward complement
            
            length = (stop_coor-start_coor)/3
            start = start_coor
            stop = stop_coor

        elif start_coor > stop_coor:
            complement = "1" #Reverse complement
            length = (-stop_coor+start_coor)/3
            start = stop_coor
            stop = start_coor

        else:
            error = "Y"
        
        
        
        

        #Check if the length of target (blast hit) is significantly smaller than query
        if length < query_length - 0.2*query_length:
            error = "Y"

        old_trans = ''
#             print(start, stop)


#Check for Met or STOP at the start and end
        while True and (start != 0 or stop != 0):
            if (query_name == query_name_list[0] or query_name == query_name_list[-1]):
                frame = 1
                translated_sequence = (sequence_extractor(species, scaff, int(complement), start, stop,frame ))

                if translated_sequence == old_trans:
                    break
                    
#Check for Met at the beginning of first exon

                if query_name == query_name_list[0] :
                    if translated_sequence[0] != "M":
                        print(translated_sequence)
#                             print(start, stop)
                        if complement == "0":
                            start = int(start) - 3
                        if complement == "1":
                            stop = int(stop) + 3
                        old_trans = translated_sequence
                    if "*" in translated_sequence:
                        error = "Y"
                        break
                    if translated_sequence[0] == "M":
                        break

#Check for stop at the end of last exon

                if query_name == query_name_list[-1]:
                    if translated_sequence[-1] != "*":
                        print(translated_sequence)
#                             print(start, stop)
                        if complement == "0":
                            stop = int(stop) + 3
                        if complement == "1":
                            start = int(start) - 3
                        old_trans = translated_sequence
                    if translated_sequence[-1] == "*":

                        break
            else:
                break
        
        
#Add offset at the beginning and end of each exon
        if (start != 0 or stop != 0):
            
            start_modifier = int(query_name.split("Frame")[1][1])
            stop_modifier = int(query_name.split("rightoh")[1][1])
        else:
            start_modifier = 0
            stop_modifier = 0            

        seq_length = query_length
        
        
#Get exon start and end by adding or removing codons
        
    #if not the beginning
        if query_start_coor != "1" and query_name != query_name_list[0]:
            if complement == "0":
                start = int(start) - 3*(int(query_start_coor)-1)                
            if complement == "1":
                stop = int(stop) + 3*(int(query_start_coor)-1)
    
    #For the end
        if query_stop_coor != str(seq_length) and query_name != query_name_list[-1]:
            if complement == "0":
                stop = int(stop) + 3*(int(seq_length)-int(query_stop_coor))
            if complement == "1":
                

                start = int(start) - 3*(int(seq_length)-int(query_stop_coor))
 
        
        
        
#Adding or removing 3' and 5' overhangs for forward and reverse complement
    #For forward complement
        if complement == "0":
            start = int(start) - int(start_modifier)
            stop = int(stop) +  int(stop_modifier)
            if old_end != 0 and old_end > stop:

                error = "Y"
            old_end = stop

    #For reverse complement
        if complement == "1":
            start = int(start) - int(stop_modifier)
            stop = int(stop) +  int(start_modifier)
            if old_end != 0 and old_end < stop:
                error = "Y"
            old_end = stop

    #Simple check for lenghth
        if start == 0 or stop == 0:
            error = "Y"
            
           
        output_format = str(species.split("\n")[0])+"," + str(scaff) +"," + str(start)+"," + str(stop)+"," + str(complement)+"," + str(error)+  ","+ str(query_name)+","+ str(query_start_coor)+","+str(query_stop_coor)+","+str(query_length)+ "\n"  
        print(output_format)
#         assert False
        Output_Sequence = Output_Sequence + output_format

    print(Output_Sequence)
    output_file = open(f"{blast_location}/1.Blast_result/{species}/{species}_coordinates.csv",'w')
    output_file.write(Output_Sequence)
    output_file.close()


Bicyclus_anynana
Bicyclus_anynana,NC_069110.1,7826744,7826832,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_0_rightoh_2_query_Exon_1,1,29,29

Bicyclus_anynana
Bicyclus_anynana,NC_069110.1,7833163,7833234,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_1_rightoh_2_query_Exon_2,1,23,23

Bicyclus_anynana
Bicyclus_anynana,NC_069110.1,7833419,7833594,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_1_rightoh_1_query_Exon_3,1,58,58

Bicyclus_anynana
Bicyclus_anynana,NC_069110.1,7835512,7835597,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_2_rightoh_0_query_Exon_4,1,28,28

Bicyclus_anynana
Bicyclus_anynana,NC_069110.1,7835941,7836009,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_0_rightoh_0_query_Exon_5,1,23,23

Bicyclus_anynana
Bicyclus_anynana,NC_069110.1,7836902,7837033,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_0_rightoh_0_query_Exon_6,1,44,44

Bicyclus_anynana
Bicyclus_anynana,NC_069110.1,7839243,7839330,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_0_rightoh_1_query_Exon_7,1,29,29

Bicyclus_anynana
Bic

NotADirectoryError: [Errno 20] Not a directory: '/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/3.Satyrine/1.Blast_result/desktop.ini/desktop.ini_blast_out.txt'